# Hiperparaméterek optimalizációja

A paraméteroptimalizációhoz a Bayesi optimalizációs technikát alkalmazom. Ehhez meg kell hívni a szükséges könyvtárat.

In [1]:
#uncomment the next line if the package is not yet installed in your machine
#!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import sys
sys.path.append('c:\\users\\ifjto\\appdata\\local\\programs\\python\\python37\\lib\\site-packages')
import numpy as np
import import_ipynb
from model import *
import torch
from torch import nn
from torch.utils.data import *
import torch.optim as optim
import torch.cuda
import torchvision.transforms as transforms
import cv2
import os
import time
from IPython.display import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

importing Jupyter notebook from model.ipynb


Az optimalizálandó paraméterek értékkészletét állítsuk be. Természetesen nem megengedhetőek az óriási értékek, mivel nincs rá elegendő számítási kapacitás - áldozatot kell hoznunk. Ezen persze segíthetünk azzal, hogy több kicsi optimalizálást futtatunk, és megpróbáljuk kitalálni, hogy melyik paraméterértékek túl kicsik vagy nagyok, így szűkítve az optimalizáló értékkészletét. Néhány magyarázat a választott határokhoz:

In [2]:
# the maximum number of pictures included in each epoch
TRAIN_SIZE = 5000
AUTO_LR = 1e-3
DISC_LR = 1e-4
NUM_EPOCH = 10

pbounds = {'nFeat' : (3, 15),
           'dropout' : (0, 0.6),
           'auto_lr' : (-7, -2),
           'disc_lr' : (-7, -4),
           'ratio' : (0.2, 0.8),
           'steps_without_disc' : (2, 8),
           'decay' : (-9, -3)}

Olvassuk be az adatokat egyszer, hogy ne kelljen minden hiperparaméterkombinációra megint.

In [3]:
trainloader, testloader, validationloader, discriminator_dataset, disc_loader = get_data(32, TRAIN_SIZE)

Epoch size: 1000
10936 images loaded


Készítsünk egy függvényt, amit az optimalizálónk tud használni. A függvény felelőssége legyen, hogy csak elfogadható értékekkel híva a train-t.

In [4]:
def test_parameters(auto_lr, disc_lr, ratio, dropout, steps_without_disc, decay, nFeat):
    global trainloader, testloader, discriminator_dataset, disc_loader
    NUM_EPOCH = 20
    auto_lr = pow(10, auto_lr)
    disc_lr = pow(10, disc_lr)
    steps_without_disc = int(round(steps_without_disc))
    decay = pow(10, decay)
    
    # redirecting stdout to avoid disturbing prints
    original_out = sys.stdout
    sys.stdout = open("temp.txt", "a")
    
    # building network
    autoencoder = Autoencoder(dropout)
    auto_optimizer = optim.Adam(autoencoder.parameters(), lr=AUTO_LR)
    auto_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(auto_optimizer, NUM_EPOCH)
    
    discriminator = Discriminator()
    disc_optimizer = optim.Adam(discriminator.parameters(), lr=DISC_LR)
    disc_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(disc_optimizer, NUM_EPOCH)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        autoencoder = autoencoder.cuda()
        discriminator = discriminator.cuda()
    
    # training network
    train(autoencoder, discriminator, trainloader, discriminator_dataset, disc_loader, NUM_EPOCH, ratio=ratio, 
          steps_without_disc=steps_without_disc, TRAIN_SIZE=TRAIN_SIZE, AUTO_LR=auto_lr, DISC_LR=disc_lr, wd=decay)
    
    result = get_result(autoencoder, testloader)
    
    # resetting the stdout
    sys.stdout = original_out
    return result

Optimalizáljunk. A jó eredményhez kezdjük néhány random lépéssel a próbálkozást.

In [5]:
start_time = time.time()
optimizer = BayesianOptimization(
    f=test_parameters,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=5,
    n_iter=20,
)

print(optimizer.max)
print('Finished after ', round(time.time() - start_time), " seconds")

|   iter    |  target   |  auto_lr  |   decay   |  disc_lr  |  dropout  |   nFeat   |   ratio   | steps_... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.141    | -4.915    | -4.678    | -7.0      |  0.1814   |  4.761    |  0.2554   |  3.118    |
|  2        |  0.1428   | -5.272    | -6.619    | -5.384    |  0.2515   |  11.22    |  0.3227   |  7.269    |
|  3        |  0.1436   | -6.863    | -4.977    | -5.748    |  0.3352   |  4.685    |  0.3189   |  6.804    |
|  4        |  0.06711  | -2.159    | -7.119    | -4.923    |  0.5258   |  13.74    |  0.251    |  2.234    |
|  5        |  0.1459   | -6.151    | -3.731    | -6.705    |  0.2527   |  14.49    |  0.5199   |  6.151    |
|  6        |  0.06711  | -2.0      | -3.0      | -7.0      |  5.551e-1 |  3.0      |  0.8      |  8.0      |
|  7        |  0.1443   | -7.0      | -9.0      | -7.0      |  0.0      |  3.0      |  0.8      |  8.0      |
|  8      